In [20]:
import sys
sys.path.append("/home/wanxinli/EHR-OT/")

from ast import literal_eval
import collections
from mimic_common import *
import os
import pandas as pd
from statistics import mean, stdev

In [21]:
def read_df(group_name, group_1, group_2, suffix = None):
    """ 
    Absoluate bound for filtering out ratios
    """
    target_equity_path = os.path.join(mimic_output_dir, f"exp4_{group_name}_{group_2}2{group_1}_equity.csv")
    if suffix is not None:
        target_equity_path = os.path.join(mimic_output_dir, f"exp4_{group_name}_{group_2}2{group_1}_{suffix}_equity.csv")
    target_equity_df = pd.read_csv(target_equity_path, header=0, index_col=None, converters={'target_codes': literal_eval})

    return target_equity_df

In [22]:
def find_outlier_codes(df):
    """ 
    :param Dataframe df: the read in dataframe

    Find outlier admissions' codes
    """
    diff_percents = list(df['target_diff_percent'])
    thres = mean(diff_percents)+3*stdev(diff_percents)

    outlier_codes = []
    for _, row in df.iterrows():
        if abs(row['target_diff_percent']) > thres:
            outlier_codes.append(row['target_codes'])
    return outlier_codes
    

In [23]:
def determine_code_freq(outlier_codes, occur_percent = 0.2):
    """ 
    Determine code frequencies for codes in outlier_codes
    :param float occur_percent: the percentage out of all outlier admissions to be classified as outlier code
    """
    num_adm = len(outlier_codes)
    outlier_codes = [code for code_vec in outlier_codes for code in code_vec]
    code_freq = collections.Counter(outlier_codes)
    code_freq = dict(sorted(code_freq.items(), key=lambda item: item[1], reverse=True))

    code_freq = {key: value for key, value in code_freq.items() if value > occur_percent*num_adm}
    return code_freq


In [24]:
def entire_proc(group_name, group_1, group_2):
    df = read_df(group_name, group_1, group_2)
    outlier_codes = find_outlier_codes(df)
    code_freq = determine_code_freq(outlier_codes)
    for code, freq in code_freq.items():
        chapter_num, chapter_name = decide_ICD_chapter(code)
        title = get_code_title(code)
        print(f"ICD code: {code}, chapter number: {chapter_num}, chapter name: {chapter_name}, code title: {title}")


In [25]:
suffix = None
group_name = 'insurance'
group_1 = 'Private'   
group_2 =  'Self_Pay'
entire_proc(group_name, group_1, group_2)

ICD code: 807.09, chapter number: 17, chapter name: Injury and Poisoning, code title: Fx mult ribs NOS-closed
ICD code: 861.21, chapter number: 17, chapter name: Injury and Poisoning, code title: Lung contusion-closed
ICD code: 427.5, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Cardiac arrest
ICD code: E849.5, chapter number: 18, chapter name: Supplementary Classification of External Causes of Injury and Poisoning, code title: Accid on street/highway
ICD code: 864.14, chapter number: 17, chapter name: Injury and Poisoning, code title: Liver lacerat, major-opn
ICD code: 276.2, chapter number: 3, chapter name: Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders, code title: Acidosis
ICD code: 458.9, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Hypotension NOS
ICD code: 300.00, chapter number: 5, chapter name: Mental Disorders, code title: Anxiety state NOS
ICD code: 305.00, chapter number: 5, chapt

In [26]:
suffix = None
group_name = 'insurance'
group_1 = 'Government'   
group_2 =  'Self_Pay'
entire_proc(group_name, group_1, group_2)

ICD code: 807.09, chapter number: 17, chapter name: Injury and Poisoning, code title: Fx mult ribs NOS-closed
ICD code: 427.5, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Cardiac arrest
ICD code: 868.03, chapter number: 17, chapter name: Injury and Poisoning, code title: Peritoneum injury-closed
ICD code: 276.2, chapter number: 3, chapter name: Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders, code title: Acidosis
ICD code: 861.21, chapter number: 17, chapter name: Injury and Poisoning, code title: Lung contusion-closed
ICD code: E849.5, chapter number: 18, chapter name: Supplementary Classification of External Causes of Injury and Poisoning, code title: Accid on street/highway
ICD code: 458.9, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Hypotension NOS
ICD code: 348.1, chapter number: 6, chapter name: Diseases of the Nervous System and Sense Organs, code title: Anoxic brain damage


In [27]:
suffix = None
group_name = 'insurance'
group_1 = 'Medicare'   
group_2 =  'Self_Pay'
entire_proc(group_name, group_1, group_2)

ICD code: 861.21, chapter number: 17, chapter name: Injury and Poisoning, code title: Lung contusion-closed
ICD code: 276.2, chapter number: 3, chapter name: Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders, code title: Acidosis
ICD code: 427.5, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Cardiac arrest
ICD code: 807.09, chapter number: 17, chapter name: Injury and Poisoning, code title: Fx mult ribs NOS-closed
ICD code: 458.9, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Hypotension NOS
ICD code: 864.14, chapter number: 17, chapter name: Injury and Poisoning, code title: Liver lacerat, major-opn
ICD code: 285.1, chapter number: 4, chapter name: Diseases of the Blood and Blood-forming Organs, code title: Ac posthemorrhag anemia
ICD code: 348.1, chapter number: 6, chapter name: Diseases of the Nervous System and Sense Organs, code title: Anoxic brain damage
ICD code: 300.00, chapter number: 5, 

In [28]:
suffix = None
group_name = 'insurance'
group_1 = 'Medicaid'   
group_2 =  'Self_Pay'
entire_proc(group_name, group_1, group_2)

ICD code: 861.21, chapter number: 17, chapter name: Injury and Poisoning, code title: Lung contusion-closed
ICD code: 807.09, chapter number: 17, chapter name: Injury and Poisoning, code title: Fx mult ribs NOS-closed
ICD code: 276.2, chapter number: 3, chapter name: Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders, code title: Acidosis
ICD code: 427.5, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Cardiac arrest
ICD code: 868.03, chapter number: 17, chapter name: Injury and Poisoning, code title: Peritoneum injury-closed
ICD code: 864.05, chapter number: 17, chapter name: Injury and Poisoning, code title: Liver lacerat unspcf cls
ICD code: 802.4, chapter number: 17, chapter name: Injury and Poisoning, code title: Fx malar/maxillary-close
ICD code: 305.00, chapter number: 5, chapter name: Mental Disorders, code title: Alcohol abuse-unspec
ICD code: E849.5, chapter number: 18, chapter name: Supplementary Classification of External 